In [ ]:
!pip install afinn
!python -m textblob.download_corpora
!pip install -U textblob
!pip install vaderSentiment
!pip install pyemd
!pip install unidecode

     |████████████████████████████████| 52 kB 1.0 MB/s 
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53447 sha256=0a668016f1b18d46921b8cb61dc64fbb9313f4db7043c48584b195e00862add6
  Stored in directory: /root/.cache/pip/wheels/9d/16/3a/9f0953027434eab5dadf3f33ab3298fa95afa8292fcf7aba75
Successfully built afinn
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
from unidecode import unidecode
import pandas as pd
import bz2
import gensim
import warnings
import numpy as np
from gensim.models import word2vec
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
from tqdm._tqdm_notebook import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

warnings.filterwarnings('ignore')
tqdm_notebook.pandas()

import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`


# Carregando os embeddings

Aqui vamos utilizar os embeddings para realizar as seguintes atividades:

- análise de simlaridade
- classificação de documentos

<b> Carregue os embeddings treinados, como vimos na Aula 2. É o mesmo arquivo que iremos utilizar</b>

Link: https://drive.google.com/open?id=1zI8pGfbUHuU_0wY_FV4tD6w6ZCUJTQbh

In [ ]:
%%time
from google.colab import drive
drive.mount('/content/drive')

# carregar
##na variável path coloque o caminho do embedding baixado:
path = "/content/drive/MyDrive/Pós-Graduação/PLN/ptwiki_20180420_100d.txt.bz2"
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(path, binary=False)

Mounted at /content/drive
CPU times: user 2min 34s, sys: 1.29 s, total: 2min 35s
Wall time: 3min 25s


# Similaridade de Documentos

Para realizar a similaridade entre documentos, utilize as frases abaixo:

In [ ]:
frase1 = "Excelente produto chegou antes do prazo indico e recomendo produto bom pois já testei e foi mais que aprovado" 
frase2 = "SUPER RECOMENDO, PREÇO, QUALIDADE #BRASTEMP, EFICIÊNCIA NA ENTREGA, E FACILIDADE DE PAGAMENTO. MUITO BOM!!!"
frase3 = "A tampa do fogão veio com problemas com o pino de encaixe solto e precisa de reparos"
frase4 = "Fogão ótimo!"

## Distância de Jaccard

<b> Atividade </b>

1) Faça um método que calcule a similaridade de Jaccard e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: lembrando que você precisa aplicar um pre-processamento nessas frases antes de aplicar o método.
Faça:

- Lower
- Remoção StopWords
- Remoção Pontuação
- Tokenização

In [ ]:
def pre_processamento_texto(corpus):
  print("Documento")
  print("#tokenizacao")
  corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]",corpus)
  #lowcase
  corpus_alt = [t.lower() for t in corpus_alt]
  print("remove stopwords")
  portuguese_stops = stopwords.words('portuguese')
  corpus_alt = [t for t in corpus_alt if t not in portuguese_stops]
  print("remove pontuação")
  corpus_alt = [t for t in corpus_alt if t not in string.punctuation]

  return corpus_alt

In [ ]:
frase1_limpa = pre_processamento_texto(frase1)
frase2_limpa = pre_processamento_texto(frase2)
frase3_limpa = pre_processamento_texto(frase3)
frase4_limpa = pre_processamento_texto(frase4)

Documento
#tokenizacao
remove stopwords
remove pontuação
Documento
#tokenizacao
remove stopwords
remove pontuação
Documento
#tokenizacao
remove stopwords
remove pontuação
Documento
#tokenizacao
remove stopwords
remove pontuação


In [ ]:
def jacard(f1,f2):
  return len(set(f1).intersection(set(f2)))/len(set(f1).union(set(f2)))

In [ ]:
print("Similiaridade Jacard:")
print("Frase1 e Frase2: ", jacard(frase1_limpa,frase2_limpa))
print("Frase1 e Frase3: ", jacard(frase1_limpa,frase3_limpa))
print("Frase2 e Frase3: ", jacard(frase2_limpa,frase3_limpa))
print("Frase1 e Frase1: ", jacard(frase1_limpa,frase4_limpa))


Similiaridade Jacard:
Frase1 e Frase2:  0.10526315789473684
Frase1 e Frase3:  0.0
Frase2 e Frase3:  0.0
Frase1 e Frase1:  0.0


<b> Atividade </b>

2) Qual par de frase teve maior simlaridade? E qual teve menor? Este resultado faz sentido? Explique.

Segundo o método de Jacard apenas as frases 1 e 2 tem similaridade pois possuem palavras que se repetem em ambas as frases. Apesar de todas as frases estarem falando sobre a opinião de um produto, pelo fato de algumas não terem palavras que aparecem nas frases comparadas, o método de Jacard indica semelhança 0.
 

## Distância de Cosseno

Aqui iremos calcular a distância do cosseno utilizando duas formas, que aprendemos na aula passada, para representar o texto.

- Bag of Words (BOW) 
- Embedding

Observação:

Existem duas formas de trabalhar com o cosseno:

<b> Distância </b>: quanto menor mais perto estão as frases.
<b> Similaridade </b>: quanto maior mais perto estão as frases.

### BOW - Distância do cosseno

<b> Atividade </b>

3) Calcule a distância do cosseno utilizando a representação CountVectorizer e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: no CountVectorizer utilizem as frases já pre-processadas da atividade anterior. Mas para aplicá-las no fit_transform, cada frase deve ser um string (sem estar tokenizada) dentro de uma lista.

```python
#exemplo
distance.cosine(frase1, frase2)
```

In [ ]:
vect_bag = CountVectorizer()

In [ ]:
docs_bag = vect_bag.fit_transform([' '.join(frase1_limpa), \
                                   ' '.join(frase2_limpa), \
                                   ' '.join(frase3_limpa), \
                                   ' '.join(frase4_limpa)])
docs_bag

<4x29 sparse matrix of type '<class 'numpy.int64'>'
	with 32 stored elements in Compressed Sparse Row format>

In [ ]:
vect_bag.get_feature_names()

['antes',
 'aprovado',
 'bom',
 'brastemp',
 'chegou',
 'eficiência',
 'encaixe',
 'entrega',
 'excelente',
 'facilidade',
 'fogão',
 'indico',
 'pagamento',
 'pino',
 'pois',
 'prazo',
 'precisa',
 'preço',
 'problemas',
 'produto',
 'qualidade',
 'recomendo',
 'reparos',
 'solto',
 'super',
 'tampa',
 'testei',
 'veio',
 'ótimo']

In [ ]:
frase1_bow = docs_bag.todense()[0]
frase2_bow = docs_bag.todense()[1]
frase3_bow = docs_bag.todense()[2]
frase4_bow = docs_bag.todense()[3]

In [ ]:
print("BOW - Distância do cosseno:")
print("Frase1 e Frase2:",distance.cosine(frase1_bow, frase2_bow))
print("Frase1 e Frase3:",distance.cosine(frase1_bow, frase3_bow))
print("Frase2 e Frase3:",distance.cosine(frase2_bow, frase3_bow))
print("Frase1 e Frase4:",distance.cosine(frase1_bow, frase4_bow))

BOW - Distância do cosseno:
Frase1 e Frase2: 0.8309691490542968
Frase1 e Frase3: 1.0
Frase2 e Frase3: 1.0
Frase1 e Frase4: 1.0


### Embedding - Distância do cosseno

Para calcular o embedding de cada uma das frases, utilize o modelo carregado inicialmente. 

Cada palavra tem um vetor, para formar o embedding da frase tire a média de todos os vetores.

Utilize as frases já pre-processadas

<b> Atividade </b> 

4) Calcule a distância do cosseno utilizando a representação Embedding e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4


In [ ]:
def calcula_embeddings(frase_tokens):
  return np.mean(np.array([word_vectors[palavra] for palavra in frase_tokens]),axis=0)

In [ ]:
frase1_emb = calcula_embeddings(frase1_limpa)
frase2_emb = calcula_embeddings(frase2_limpa)
frase3_emb = calcula_embeddings(frase3_limpa)
frase4_emb = calcula_embeddings(frase4_limpa)

In [ ]:
print("EMBEDDING - Distância do cosseno:")
print("Frase1 e Frase2:",distance.cosine(frase1_emb, frase2_emb))
print("Frase1 e Frase3:",distance.cosine(frase1_emb, frase3_emb))
print("Frase2 e Frase3:",distance.cosine(frase2_emb, frase3_emb))
print("Frase1 e Frase4:",distance.cosine(frase1_emb, frase4_emb))

EMBEDDING - Distância do cosseno:
Frase1 e Frase2: 0.16096973419189453
Frase1 e Frase3: 0.2234203815460205
Frase2 e Frase3: 0.2526938319206238
Frase1 e Frase4: 0.29043054580688477


<b>Atividade </b>

5) Qual diferença entre as representações do exercício 3 e 4? Qual faz mais sentido? Explique.

O Embedding leva em consideração o contexto e as possíveis dimensões em que cada palavra pode ser colocada. Como as frases foram obtidas das avaliações de fogão, de certo modo o Embedding entende essa proximidade das palavras. O BOW leva em consideração apenas as palavras, portanto pela própria natureza do método não é possível que o computador "entenda" o contexto das frases e com qual sentido elas foram usadas.


## WMD

O WMD já está incorporado ao Word2Vec

<b> Atividade </b>

6) Calcule a distância WMD e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: use a variável já tokenizada.

In [ ]:
print("WMD:")
print("Frase1 e Frase2: ",word_vectors.wmdistance(frase1_limpa,frase2_limpa))
print("Frase1 e Frase3: ",word_vectors.wmdistance(frase1_limpa,frase3_limpa))
print("Frase2 e Frase3: ",word_vectors.wmdistance(frase2_limpa,frase3_limpa))
print("Frase1 e Frase4: ",word_vectors.wmdistance(frase1_limpa,frase4_limpa))

WMD:
Frase1 e Frase2:  3.0725697404938526
Frase1 e Frase3:  3.759219170455064
Frase2 e Frase3:  3.9048993635316385
Frase1 e Frase4:  3.851623338336251


<b> Atividade </b>

7) Qual par de frase teve maior distância? E qual teve menor? Este resultado faz sentido? Explique.

As frases 2 e 3 tem a maior distância. As frases com menor distância foram a 1 e 2. Outros métodos já haviam demonstrado que as frases 1 e 2 eram as mais semelhantes, portanto o resultado já era de certo modo esperado. O interessante na utilização do WMD é que se pararmos pra perceber, numericamente o valor da distância das outras frases sem ser a 1 e 2 não é tão grande.

# Classificação de Documentos

A clssificação de documentos é muito útil em vários aspectos. Um dos tipos de classificação de texto é a análise de sentimentos.

A fim de ilustrar a classificação de documentos iremos criar um modelo para classificar uma frase como positiva ou negativa.

## Dataset

<b> Atividade </b>

8) Carregue o dataset com o pandas e depois dê o head no dataframe.


Link download: https://drive.google.com/open?id=15azJWdEEPGsXQGiDmEOseTBJcquWvBQc

<b> Este dataset é sobre revisões de filmes do IMDB. </b>

In [ ]:
path1 = "/content/drive/MyDrive/Pós-Graduação/PLN/imdb-reviews-pt-br.csv"
df = pd.read_csv(path1)
df

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg
...,...,...,...,...
49454,49456,"Seeing as the vote average was pretty low, and...","Como a média de votos era muito baixa, e o fat...",pos
49455,49457,"The plot had some wretched, unbelievable twist...",O enredo teve algumas reviravoltas infelizes e...,pos
49456,49458,I am amazed at how this movieand most others h...,Estou espantado com a forma como este filme e ...,pos
49457,49459,A Christmas Together actually came before my t...,A Christmas Together realmente veio antes do m...,pos


In [ ]:
df['sentiment'].value_counts()

neg    24765
pos    24694
Name: sentiment, dtype: int64

## Representação dos dados

O sentimento positivo e negativo iremos binarizar cada um deles. Seja 1 positivo e 0 negativo.

Iremos representar o texto de duas formas:

- Bag of Words (BOW)
- Embedding

Depois iremos comparar o resultado de cada um deles.

### Representação Target

<b> Atividade </b>

9) Faça a representação dos sentimentos. 1 positivo; 0 negativo

In [ ]:
target = df['sentiment'].replace(['neg','pos'],[0,1])
target

0        0
1        0
2        0
3        0
4        0
        ..
49454    1
49455    1
49456    1
49457    1
49458    1
Name: sentiment, Length: 49459, dtype: int64

### Bag of Words (BOW)

<b> Atividade </b>

10) Aplique o pré-processamento listado abaixo na coluna ``text_pt`` (crie uma nova coluna ```text_pt_sem_stopwords``` no dataframe para armazenar este dado processado):

- Remova as stopwords do texto
- Remova as pontuções
- Mantenha o texto sem tokenização, ou seja uma string

<b> Dica: </b> use o ```progress_apply``` para exibir a barra de progresso:

```python
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
df["colunas"].progress_apply(lambda x: preprocessamento(x))
```

In [ ]:
def pre_processamento_texto_str(corpus):
  #print("#tokenizacao")
  corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]",corpus)
  #print("remove stopwords")
  portuguese_stops = stopwords.words('portuguese')
  corpus_alt = [t for t in corpus_alt if t not in portuguese_stops]
  #print("remove numeros")
  corpus_alt = [re.sub(r'\d','',t) for t in corpus_alt]
  #print("remove pontuação")
  corpus_alt = [t for t in corpus_alt if t not in string.punctuation]

  corpus_alt_str = ' '.join(corpus_alt).lower()

  return corpus_alt_str

In [ ]:
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
df["text_pt_limpo"] = df["text_pt"].progress_apply(lambda x: pre_processamento_texto_str(x))

  0%|          | 0/49459 [00:00<?, ?it/s]

In [ ]:
df["text_pt_limpo"]

0        mais vez sr costner arrumou filme tempo necess...
1        este exemplo motivo maioria filmes ação mesmos...
2        primeiro tudo odeio raps imbecis poderiam agir...
3        nem beatles puderam escrever músicas todos gos...
4        filmes fotos latão palavra apropriada verdade ...
                               ...                        
49454    como média votos baixa fato funcionário locado...
49455    o enredo algumas reviravoltas infelizes inacre...
49456    estou espantado forma filme maioria outros méd...
49457    a christmas together realmente veio antes temp...
49458    o drama romântico classe trabalhadora diretor ...
Name: text_pt_limpo, Length: 49459, dtype: object

<b> Atividade </b>

11) Aplique a representação do texto processado anteriormente com CountVectorizer.

In [ ]:
vect_bag = CountVectorizer()

In [ ]:
x_bag = vect_bag.fit_transform(df['text_pt_limpo'])

In [ ]:
x_bag

<49459x127624 sparse matrix of type '<class 'numpy.int64'>'
	with 5267081 stored elements in Compressed Sparse Row format>

### Embedding

<b> Atividade </b>

12) Aplique o pré-processamento listado abaixo na coluna ``text_pt`` (crie uma nova coluna ```text_pt_sem_stopwords_token``` no dataframe para armazenar este dado processado):

- Aplique lower
- Remova as stopwords do texto
- Remova as pontuções
- Mantenha o texto com tokenização

<b> Dica: </b> use o ```progress_apply``` para exibir a barra de progresso:

```python
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
df["colunas"].progress_apply(lambda x: preprocessamento(x))
```

In [ ]:
def pre_processamento_texto_token(corpus):
  #print("Documento")
  #print("#tokenizacao")
  corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]",corpus)
  #lowcase
  corpus_alt = [t.lower() for t in corpus_alt]
  #print("remove stopwords")
  portuguese_stops = stopwords.words('portuguese')
  corpus_alt = [t for t in corpus_alt if t not in portuguese_stops]
  #print("remove numeros")
  corpus_alt = [re.sub(r'\d','',t) for t in corpus_alt]
  #print("remove pontuação")
  corpus_alt = [t for t in corpus_alt if t not in string.punctuation]
  return corpus_alt

In [ ]:
df["text_pt_sem_stopwords_token"] = df['text_pt'].progress_apply(lambda x: pre_processamento_texto_token(x))

  0%|          | 0/49459 [00:00<?, ?it/s]


<b> Atividade </b>

13) Aplique a representação do texto com Embeddings. Cada palavra tem um embedding, o embedding da frase é a média de todos embeddings.

In [ ]:
def calcula_embeddings_frase(tokens):
  return np.mean([word_vectors[t] for t in tokens if t in word_vectors.vocab],axis=0)

In [ ]:
x_embedding = df['text_pt_sem_stopwords_token'].progress_apply(lambda x: calcula_embeddings_frase(x ))

  0%|          | 0/49459 [00:00<?, ?it/s]

In [ ]:
x_embedding

0        [0.14301138, 0.22025318, -0.047075942, -0.2170...
1        [0.16303213, 0.22678684, -0.05451168, -0.20547...
2        [0.21659917, 0.14430527, -0.0034280657, -0.197...
3        [0.2596114, 0.13605723, -0.07343373, -0.208256...
4        [0.22052252, 0.087914094, -0.06986406, -0.1659...
                               ...                        
49454    [0.16023268, 0.1474827, -0.08474423, -0.103290...
49455    [0.17767419, 0.1423968, -0.081170976, -0.22615...
49456    [0.18305564, 0.18313898, -0.062747374, -0.1887...
49457    [0.18229564, 0.15024172, -0.08812309, -0.22163...
49458    [0.26787597, 0.16923124, -0.06144687, -0.18159...
Name: text_pt_sem_stopwords_token, Length: 49459, dtype: object

## Treinamento

### CountVectorizer


<b> Atividade </b>

14) Faça a divisão dados dados em treino e teste como no exemplo abaixo:

```python
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bag, target,random_state=123)
```

In [ ]:
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(x_bag, target,random_state=123)


<b> Atividade </b>

15) Treine com uma regressão logística

In [ ]:
modelo_bow = LogisticRegression()
modelo_bow.fit(X_train_bow,y_train_bow)

LogisticRegression()


<b> Atividade </b>

16) Calcule as métricas de resultado utilizando método abaixo:

```python
print(classification_report(y_test_bow, y_pred))
```

In [ ]:
y_pred = modelo_bow.predict(X_test_bow)

In [ ]:
print(classification_report(y_test_bow, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.87      0.88      6112
           1       0.87      0.89      0.88      6253

    accuracy                           0.88     12365
   macro avg       0.88      0.88      0.88     12365
weighted avg       0.88      0.88      0.88     12365



### Embedding


<b> Atividade </b>

17) Faça a divisão dados dados em treino e teste como no exemplo abaixo:

Verifique o shape do X treino e X teste. Caso eles estejam com apenas uma dimensão, você precisa tranformá-los para duas dimensões, caso contrário ocorrerá erro no treinamento.

In [ ]:
X_train_embedding, X_test_embedding, y_train_embedding, y_test_embedding = train_test_split(x_embedding.values,target,random_state=123)

In [ ]:
X_train_embedding = pd.DataFrame([x for x in X_train_embedding])
X_train_embedding.shape

(37094, 100)

In [ ]:
X_test_embedding = pd.DataFrame([x for x in X_test_embedding])
X_test_embedding.shape

(12365, 100)


<b> Atividade </b>

18) Treine com uma regressão logística

In [ ]:
modelo_embedding = LogisticRegression()
modelo_embedding.fit(X_train_embedding,y_train_embedding)

LogisticRegression()


<b> Atividade </b>

19) Calcule as métricas de resultado utilizando método abaixo:

```python
print(classification_report(y_test_bow, y_pred))
```

#### Calcule as métricas

In [ ]:
y_pred = modelo_embedding.predict(X_test_embedding)

In [ ]:
print(classification_report(y_test_embedding, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.77      0.77      6112
           1       0.78      0.77      0.77      6253

    accuracy                           0.77     12365
   macro avg       0.77      0.77      0.77     12365
weighted avg       0.77      0.77      0.77     12365




<b> Atividade </b>

20) Compare os resultados obtidos com o BagOfWords e com o Embedding. Explique os possíveis motivos desta diferença.

O BOW teve as melhores métricas. No meu ponto de vista o BOW foi melhor que o modelo de Embedding porque o modelo foi mais simples e tinha mais caracteristicas para serem utilizadas. O Embedding utilizou como modelo 100 caracteristicas que representavam as 100 dimensões com valores decimais reais. Enquanto o BOW utilizou 127624 caracteristicas para criar o modelo, sendo os valores 0 e 1.

# Análise de sentimentos

O modelo que criamos anteriormente é para ilustrar como podemos realizar classificação de documentos.
Quando a tarefa é sobre análise de sentimentos, temos duas opções: treinar nosso próprio modelo, como feito anteriormente ou utilizar uma das inúmeras ferramentas prontas.

Vamos testar as seguintes ferramentas:

- Vader
- Textblob
- Affin

Nesta atividade iremos utilizar as duas variáveis abaixo:

In [ ]:
texto_neg = df.loc[0, "text_en"]
texto_pos = df.loc[49431, "text_en"]

In [ ]:
print(texto_neg)
print(texto_pos)

Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costners character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks hes better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutchers ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in.
SPOILERS THROUGH: I really am in the minority on this one but I liked this movie. Its not a classic bu

## Vader

<b> Apenas Inglês </b>

O VADER (Valence Aware Dictionary e sEntiment Reasoner) é uma ferramenta de análise de sentimentos baseada em regras e léxico, especificamente identifica os sentimentos expressos nas mídias sociais.

- positive sentiment: compound score >= 0.05
- neutral sentiment: (compound score > -0.05) e (compound score < 0.05)
- negative sentiment: compound score <= -0.05

Mais informações: https://github.com/cjhutto/vaderSentiment


<b> Atividade </b>

21) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(texto)
```

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
analyzer = SentimentIntensityAnalyzer()
res_neg = analyzer.polarity_scores(texto_neg)
res_pos = analyzer.polarity_scores(texto_pos)
print("VADER Negativo: ", res_neg)
print("VADER Positivo: ", res_pos)

VADER Negativo:  {'neg': 0.126, 'neu': 0.76, 'pos': 0.114, 'compound': 0.3958}
VADER Positivo:  {'neg': 0.084, 'neu': 0.737, 'pos': 0.179, 'compound': 0.9969}


## TextBlob

<b> Apenas inglês </b>

https://www.presentslide.in/2019/08/sentiment-analysis-textblob-library.html

<b> Atividade </b>
 
22) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
sentence=TextBlob(texto)
sentence.sentiment
```

In [ ]:
from textblob import TextBlob

In [ ]:
sentence=TextBlob(texto_neg)
print("TextBlob Negativo: ", sentence.sentiment)

Sentiment(polarity=0.06385964912280702, subjectivity=0.5629824561403508)

In [ ]:
sentence=TextBlob(texto_pos)
print("TextBlob Positivo: ", sentence.sentiment)

TextBlob Positivo:  Sentiment(polarity=0.190819118692253, subjectivity=0.6026226012793177)


## Afinn

- Valor maior que 0 indica sentimento positivo
- Valor menor que 0 indica sentimento negativo

<b> Atividade </b>

23) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
afinn = Afinn()
afinn.score(texto)
```

In [ ]:
from afinn import Afinn

In [ ]:
afinn = Afinn()
print("Afinn Negativo: ", afinn.score(texto_neg))
print("Afinn Positivo: ", afinn.score(texto_pos))

Afinn Negativo:  8.0
Afinn Positivo:  55.0


<b> Atividade </b>

24) Para você, qual ferramenta teve melhor comportamento?  Justifique com exemplos.

No meu ponto de vista a melhor ferramenta foi o VADER pois os resultados foram mais claros e mais compreensíveis. A própria definição de qual o sentimento do texto foi mais clara no VADER, além de mostrar informações mais detalhadas sobre os valores relativos a cada sentimento. Por exemplo mostrando quais os scores relatativos a cada categoria. No caso da frase negativa o score dela neutro era bem alto, e o compound indicando que ela é neutra.

# Dica:
## Quando for trabalhar com um dataset em inglês, a biblioteca Spacy facilita!

In [ ]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4 MB 1.2 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-py3-none-any.whl size=98051301 sha256=8eece0b96d5e4bcbf9b0074b20f3127120ff5781cc1b80e9808ab32f605c17b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-3e5r8cnc/wheels/69/c5/b8/4f1c029d89238734311b3269762ab2ee325a42da2ce8edb997
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
import spacy
import pandas as pd

O scpay forne um pacote que já tem série de modelos já treinados em NLP. Inclusive para os embeddings em inglês.

Para mais informações vá em:

https://spacy.io/models/en#en_core_web_md



Com o método abaixo carregamos um dos modelos do spacy:

```python
nlp = spacy.load('en_core_web_md')
```

Para aplicar o modelo, basta passar o texto para o modelo carregado anteriormente:

```python
doc = nlp("This is some text that I am processing with Spacy")
```

Carregue o modelo e imprima doc

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
doc = nlp("This is some text that I am processing with Spacy")

In [ ]:
doc

This is some text that I am processing with Spacy

Ao aplicar o modelo carregado a variável <b> doc </b> já possui os embeddings de cada uma das palavras e o embedding da frase, que é a média de todos vetores de todas palavras.

```python
#vetor da primeira palavra
doc[0].vector
#vetor agregado pela média - embedding do documento
doc.vector
```
O código abaixo mostra que a média de uma posição em específico dos embeddings de todas as palavras e a posição do embedding do documento possuem o mesmo valor.

In [ ]:
def calcula_media_posicao(x):
    soma = 0
    vector = []
    for i in range(0,len(doc)):
        vector.append(doc[i].vector)    
    
    for v in vector:
        soma += v[x]
    return soma/len(doc)

In [ ]:
round(calcula_media_posicao(10),6)

-0.403437

In [ ]:
round(doc.vector[10], 6)

-0.403437